<a href="https://colab.research.google.com/github/rikdantas/Aprendizagem-de-Maquinas/blob/main/IMD1101/Pratica_01/Pratica01_PCA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prática 01 - Utilização de extrator para imagens e uso do k-NN
Nesse notebook será aplicado o método do PCA nas 6 melhores bases, resultando assim em 6 novas bases.

Aluno: Paulo Ricardo Dantas

In [15]:
# Importando as bibliotecas
import numpy as np
import pandas as pd
import gdown
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

## Baixando os datasets

In [2]:
# Criando função para baixar os datasets, assim deixando o código um pouco mais organizado
def baixar_dataset(url, output):
    gdown.download(url, output, quiet=True)

In [3]:
# Baixando datasets do VGG16
baixar_dataset('https://drive.google.com/uc?id=1Lig0-UqpGvMQCmV5r5U__yucoocrlbtf', 'VGG16_1.csv')
baixar_dataset('https://drive.google.com/uc?id=1V7_N11eqBnTiM9iKY3vhMVrTzoERrijS', 'VGG16_2.csv')
baixar_dataset('https://drive.google.com/uc?id=1d8spY-o4xBsozdeSbGGlweZrips1m0HC', 'VGG16_3.csv')

# Baixando datasets do VGG19
baixar_dataset('https://drive.google.com/uc?id=1yjuqQKxXtBqlMetGgD3fyFjDZ3zg3YDS', 'VGG19_1.csv')
baixar_dataset('https://drive.google.com/uc?id=13nmb2kPrv6oVKlhodM-_ierzsMrvWL18', 'VGG19_2.csv')
baixar_dataset('https://drive.google.com/uc?id=1qB2SQXLtgS01FqTrwGYLdOY1pwRqkuXu', 'VGG19_3.csv')

## Importando os datasets
Vamos usar o pandas para importar os datasets para DataFrames. Usaremos os mesmos nomes que foram baixados os arquivos dos datasets. Podemos relacionar os datasets com as configurações como visto na tabela a seguir:

| Nome do dataset | Configuração         |
|------------------|----------------------|
| VGG16_1         | CNN_VGG16_128_avg   |
| VGG16_2         | CNN_VGG16_128_max   |
| VGG16_3         | CNN_VGG16_256_avg   |
| VGG19_1         | CNN_VGG19_128_avg   |
| VGG19_2         | CNN_VGG19_128_max   |
| VGG19_3         | CNN_VGG19_256_avg   |

In [4]:
# Importando os datasets do VGG16
vgg16_1 = pd.read_csv('VGG16_1.csv', encoding='utf-8')
vgg16_2 = pd.read_csv('VGG16_2.csv', encoding='utf-8')
vgg16_3 = pd.read_csv('VGG16_3.csv', encoding='utf-8')

# Importando os datasets do VGG19
vgg19_1 = pd.read_csv('VGG19_1.csv', encoding='utf-8')
vgg19_2 = pd.read_csv('VGG19_2.csv', encoding='utf-8')
vgg19_3 = pd.read_csv('VGG19_3.csv', encoding='utf-8')

## Aplicando o PCA

In [10]:
# Lista com os datasets já carregados e suas configurações
datasets = {
    "vgg16_1": ("CNN_VGG16_128_avg", vgg16_1),
    "vgg16_2": ("CNN_VGG16_128_max", vgg16_2),
    "vgg16_3": ("CNN_VGG16_256_avg", vgg16_3),
    "vgg19_1": ("CNN_VGG19_128_avg", vgg19_1),
    "vgg19_2": ("CNN_VGG19_128_max", vgg19_2),
    "vgg19_3": ("CNN_VGG19_256_avg", vgg19_3),
}

# Dicionário para armazenar os resultados do PCA
pca_results = {}


In [11]:
for dataset_name, (config, data) in datasets.items():
    # Separar as features e a coluna 'class'
    X = data.iloc[:, :-1]  # Todas as colunas exceto a última
    y = data["class"]      # Última coluna

    # Aplicar o PCA com 10 componentes
    pca = PCA(n_components=10)
    X_pca = pca.fit_transform(X)

    # Combinar os dados reduzidos com a coluna 'class'
    pca_data = pd.DataFrame(X_pca, columns=[f"PC{i+1}" for i in range(10)])
    pca_data["class"] = y

    # Nomear a base resultante
    pca_name = f"PCA_{config}"
    pca_results[pca_name] = pca_data

    # Salvar em CSV
    pca_data.to_csv(f"{pca_name}.csv", index=False, encoding="utf-8")

## Aplicando o kNN nas novas bases

### Importando os datasets como dataframes

In [13]:
# Importando os novos datasets resultantes do PCA
pca_vgg16_1 = pd.read_csv('PCA_CNN_VGG16_128_avg.csv', encoding='utf-8')
pca_vgg16_2 = pd.read_csv('PCA_CNN_VGG16_128_max.csv', encoding='utf-8')
pca_vgg16_3 = pd.read_csv('PCA_CNN_VGG16_256_avg.csv', encoding='utf-8')
pca_vgg19_1 = pd.read_csv('PCA_CNN_VGG19_128_avg.csv', encoding='utf-8')
pcar_vgg19_2 = pd.read_csv('PCA_CNN_VGG19_128_max.csv', encoding='utf-8')
pca_vgg19_3 = pd.read_csv('PCA_CNN_VGG19_256_avg.csv', encoding='utf-8')

### Utilizando holdout


In [14]:
# Lista de DataFrames
dataframes = {
     "pca_vgg16_1": pca_vgg16_1,
     "pca_vgg16_2": pca_vgg16_2,
     "pca_vgg16_3": pca_vgg16_3,
     "pca_vgg19_1": pca_vgg19_1,
     "pca_vgg19_2": pcar_vgg19_2,
     "pca_vgg19_3": pca_vgg19_3
}

# Configurações
results = {}
random_state = 1

In [16]:
# Loop para aplicar k-NN
for name, df in dataframes.items():
    # A última coluna é o rótulo (classe)
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]

    # Holdout (70/30)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=random_state)

    # Testar K de 1 a 10
    accuracies = []
    for k in range(1, 11):
        knn = KNeighborsClassifier(n_neighbors=k)
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        acc = accuracy_score(y_test, y_pred)
        accuracies.append(acc)

    # Armazenar resultados
    results[name] = accuracies

# Exibir resultados
results_df = pd.DataFrame(results, index=[f"K={k}" for k in range(1, 11)])
print(results_df)

      pca_vgg16_1  pca_vgg16_2  pca_vgg16_3  pca_vgg19_1  pca_vgg19_2  \
K=1      0.566667     0.595833     0.666667     0.541667     0.562500   
K=2      0.579167     0.566667     0.654167     0.570833     0.533333   
K=3      0.608333     0.595833     0.675000     0.625000     0.612500   
K=4      0.575000     0.583333     0.658333     0.637500     0.608333   
K=5      0.612500     0.616667     0.675000     0.645833     0.650000   
K=6      0.591667     0.616667     0.679167     0.629167     0.604167   
K=7      0.591667     0.633333     0.662500     0.641667     0.625000   
K=8      0.612500     0.633333     0.662500     0.629167     0.633333   
K=9      0.620833     0.637500     0.670833     0.629167     0.620833   
K=10     0.645833     0.641667     0.675000     0.629167     0.616667   

      pca_vgg19_3  
K=1      0.579167  
K=2      0.604167  
K=3      0.608333  
K=4      0.591667  
K=5      0.629167  
K=6      0.629167  
K=7      0.637500  
K=8      0.612500  
K=9      0.62500

### Utilizando K-fold


In [17]:
# Lista de DataFrames
dataframes = {
     "pca_vgg16_1": pca_vgg16_1,
     "pca_vgg16_2": pca_vgg16_2,
     "pca_vgg16_3": pca_vgg16_3,
     "pca_vgg19_1": pca_vgg19_1,
     "pca_vgg19_2": pcar_vgg19_2,
     "pca_vgg19_3": pca_vgg19_3
}

# Configurações
results = {}

In [18]:
# Loop para aplicar k-NN
for name, df in dataframes.items():
    # A última coluna seja é o rótulo (classe)
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]

    # Testar K de 1 a 10
    accuracies = []
    for k in range(1, 11):
        knn = KNeighborsClassifier(n_neighbors=k)
        # 10-fold CV
        scores = cross_val_score(knn, X, y, cv=10, scoring='accuracy')
        accuracies.append(np.mean(scores))

    # Armazenar resultados
    results[name] = accuracies

# Exibir resultados
results_df = pd.DataFrame(results, index=[f"K={k}" for k in range(1, 11)])
print(results_df)

      pca_vgg16_1  pca_vgg16_2  pca_vgg16_3  pca_vgg19_1  pca_vgg19_2  \
K=1       0.57625      0.62250      0.65125      0.59750      0.60375   
K=2       0.57375      0.60125      0.65375      0.60625      0.60625   
K=3       0.61875      0.65000      0.67750      0.62500      0.63875   
K=4       0.62375      0.62875      0.67375      0.63375      0.63625   
K=5       0.65625      0.65625      0.69375      0.64000      0.66625   
K=6       0.63125      0.64750      0.68375      0.64875      0.65375   
K=7       0.64375      0.66625      0.69500      0.63250      0.67250   
K=8       0.64625      0.66000      0.68500      0.65125      0.67500   
K=9       0.67000      0.68125      0.69625      0.65375      0.67500   
K=10      0.66250      0.67750      0.68875      0.64625      0.67375   

      pca_vgg19_3  
K=1       0.63250  
K=2       0.63250  
K=3       0.67250  
K=4       0.65625  
K=5       0.67375  
K=6       0.67500  
K=7       0.67000  
K=8       0.69000  
K=9       0.6787